In [ ]:
from DTC.dtc_executor import DTCExecutor
from DTC.route_skeleton import RouteSkeleton
import json
from math import ceil

In [ ]:
import pickle

#a = DTCExecutor()
#pc = a.create_point_cloud_with_n_points(17000000, city=True)



In [ ]:
#with open('./pc.pickle', 'wb') as handle:
#    pickle.dump(pc, file=handle)

In [ ]:
pc = None
with open('./pc.pickle', 'rb') as handle:
    pc = pickle.load(handle)

In [ ]:
from DTC.gridsystem import GridSystem
from DTC.construct_safe_area import SafeArea
gs = GridSystem(pc)
route_skeleton = set()
with open("./AllcityRSK.json", "r") as file:
    data = json.load(file)
    for elem in data:
        clean_elem = elem.strip('()').split(',')

        set_elem = tuple(float(num.strip()) for num in clean_elem)

        route_skeleton.add(set_elem)


converted_data = {}
with open("./AllcitySA.json", "r") as file:
    data = json.load(file)
    for key, value in data.items():
        
        
        clean_key = key.strip('()').split(',')
        tuple_key = tuple(float(num.strip()) for num in clean_key)

        converted_data[tuple_key] = SafeArea(tuple_key, radius=value[0], cardinality=value[1], confidence_change=1, normalisation_factor=0.5, cardinality_squish=0.15, max_confidence_change=0.15)


gs.safe_areas = converted_data
gs.route_skeleton = route_skeleton


In [ ]:
import copy
i = 0
for traj in gs.pc.trajectories:
    i+=1
#old_traj = copy.deepcopy(gs.pc.trajectories)
print(i)

In [ ]:
#extract r
import random
traj_for_test = random.sample(gs.pc.trajectories, 1467)



In [ ]:
i = 0
for traj in traj_for_test:
    for point in traj.points:
        i += 1
print(i)

In [ ]:
# use 5% of 52489 - or 5 % of all the points in the collective pointcloud for all trajectories
from math import ceil
amount_to_shift = ceil(i * 0.05)
print(amount_to_shift, " and ", i)

***We use random.sample to get a random set of trajectories every time this runs***
- furthermore, we take 5% of the points and shift them between 1-200m randomly 
- this is done to test the cleaning of these

In [ ]:
from DTC.distance_calculator import DistanceCalculator
list_of_points = []
for traj in traj_for_test:
    for point in traj.points:
        print(point.longitude)
        list_of_points.append(point)
list_of_points = copy.copy(list_of_points)

points_to_shift = random.sample(list_of_points, amount_to_shift)
print(len(points_to_shift))

In [ ]:
from DTC.distance_calculator import DistanceCalculator
from DTC.point import Point
from datetime import datetime
#Shifting the points twice seem to give me coordinate that does not work - this does not really make any sense

for point in points_to_shift:
    print(point.longitude)
    point_shifted = DistanceCalculator.shift_point_with_bearing(point, random.randint(1,200), random.randint(0,359))
    point.longitude = point_shifted[0]
    point.latitude = point_shifted[1]
    point.noise = True
    #point = DistanceCalculator.shift_point_with_bearing_and_alter_point(point, random.randint(1,200), DistanceCalculator.EAST, gs.initialization_point)

In [ ]:
for point in points_to_shift:
    print(point.longitude)

In [ ]:
from DTC import clean
clean_t = clean.CleanTraj(gs.safe_areas, gs.route_skeleton, gs.initialization_point)


In [ ]:

cleaned_points = clean_t.clean(list_of_points)

- After Cleaning, make comparison with the old trajectory points

In [ ]:
print(len(cleaned_points))
i = 0
for point in cleaned_points:
    if point == True:
        i += 1
print(i)
i = 0
for point in list_of_points:
    if point.noise == True:
        i += 1
print(i)

**Test Setup**

- Using the setup from the paper DTC
- Using 1472 trajectories as in the paper in the first set of experiments
- 



**This is only with a MR with 2million points**

*first run*
- 926 cleaned
- 3107 to clean

*second run*
- 1140 cleaned
- 2564 to clean


**Running with 11 million points**

*first run*
- 739 cleaned
- 2573 to clean

*second run*
- 775 cleaned
- 2342 to clean

*third run*
- 850 cleaned
- 2394 to clean

*fourth run*
- 656 cleaned
- 2386 to clean

*fifth run*
- 1137 cleaned
- 3998 to clean

- VERY low amount of true positives - thats bad


In [ ]:
average_cleaned = sum([739, 775, 850, 656]) / sum([2573, 2342, 2394, 2386])
print(average_cleaned)